In [12]:
from datetime import timedelta, date
import pandas
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
from datetime import datetime, timedelta
from IPython.display import display, HTML

thisdir = '/home/idies/workspace/Storage/raddick/nba/'
teamfile = thisdir + 'teamlist.csv'

outdir = '/home/idies/workspace/Temporary/raddick/nba_scratch/'

season = '2019_2020c'

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
print('ok')

ok


In [13]:
teams_df = pandas.read_csv(teamfile)
teams_df.columns = [x.lower() for x in teams_df.columns]
#teams_df[(teams_df['lg'] == 'NBA') & (teams_df['to'].apply(lambda x: int(x[0:4])) >= 2009)]
teamlist = ['ATL', 'BRK', 'NJN', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW']
teamlist += ['HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIN', 'NOP', 'NOH', 'OKC']
teamlist += ['ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

start_date = date(2019, 10, 22)
#end_date = date(2019, 6, 14)
end_date = date.today()


all_games_df = pandas.DataFrame()

all_games_meta_records = []


for thisteam in teamlist[20:]:
    print('finding games for {0:}...'.format(thisteam))
    for single_date in daterange(start_date, end_date):
        gameID = single_date.strftime("%Y%m%d")+thisteam
        theurl = 'https://www.basketball-reference.com/boxscores/{0:}0{1:}.html'.format(single_date.strftime("%Y%m%d"),thisteam)
        
        response = requests.get(theurl)
        if (response.status_code == 200):
            this_game_meta_dict = {}
            this_game_meta_dict['gameID'] = gameID
            
            print('\t'+theurl)
            soup = bs(response.content, "html.parser", from_encoding="utf-8")
            
            scorebox = soup.find('div', class_='scorebox')
            this_game_meta_dict['visiting_team'] = scorebox.findAll('strong')[0].get_text().strip()
            this_game_meta_dict['home_team'] = scorebox.findAll('strong')[1].get_text().strip()
            this_game_meta_dict['visiting_score'] = pandas.to_numeric(scorebox.findAll('div', class_='score')[0].get_text(), errors='coerce')
            this_game_meta_dict['home_score'] = pandas.to_numeric(scorebox.findAll('div', class_='score')[1].get_text(), errors='coerce')            
            
            this_game_meta_dict['visiting_record_pre_wins'] = pandas.to_numeric(scorebox.findAll('div')[5].get_text().split('-')[0], errors='coerce')
            this_game_meta_dict['visiting_record_pre_losses'] = pandas.to_numeric(scorebox.findAll('div')[5].get_text().split('-')[1], errors='coerce')
            
            this_game_meta_dict['home_record_pre_wins'] = pandas.to_numeric(scorebox.findAll('div')[12].get_text().split('-')[0], errors='coerce')
            this_game_meta_dict['home_record_pre_losses'] = pandas.to_numeric(scorebox.findAll('div')[12].get_text().split('-')[1], errors='coerce')
                                    
            # Remove result of the game under consideration to get pregame records
            if (this_game_meta_dict['visiting_score'] > this_game_meta_dict['home_score']):
                this_game_meta_dict['visiting_record_pre_wins'] = this_game_meta_dict['visiting_record_pre_wins'] - 1
                this_game_meta_dict['home_record_pre_losses'] = this_game_meta_dict['home_record_pre_losses'] - 1
            elif (this_game_meta_dict['visiting_score'] < this_game_meta_dict['home_score']):
                this_game_meta_dict['visiting_record_pre_losses'] = this_game_meta_dict['visiting_record_pre_losses'] - 1
                this_game_meta_dict['home_record_pre_wins'] = this_game_meta_dict['home_record_pre_wins'] - 1

            this_game_meta_dict['visitng_pre_win_pct'] = this_game_meta_dict['visiting_record_pre_wins'] / (this_game_meta_dict['visiting_record_pre_wins'] + this_game_meta_dict['visiting_record_pre_losses'])
            this_game_meta_dict['home_pre_win_pct'] = this_game_meta_dict['home_record_pre_wins'] / (this_game_meta_dict['home_record_pre_wins'] + this_game_meta_dict['home_record_pre_losses'])
            
            scorebox_meta_entries = soup.find('div', class_='scorebox_meta').findAll('div')
            this_game_meta_dict['datetime'] = pandas.to_datetime(scorebox_meta_entries[0].get_text())
            this_game_meta_dict['arena'] = scorebox_meta_entries[1].get_text().split(',')[0]
            this_game_meta_dict['city'] = scorebox_meta_entries[1].get_text().split(',')[1]
            this_game_meta_dict['state'] = scorebox_meta_entries[1].get_text().split(',')[2]
            
            all_games_meta_records.append(this_game_meta_dict)

            thetables = soup.findAll('table')
            
            this_game_record = []
            
            for theside in [0,8]:   # visiting team = 0, home team = 8
                
                elteam = thetables[theside].get_text()[0:thetables[theside].get_text().find("(")]
                basic_stats_this_team_rows = thetables[theside+6].findAll('tr')
                for i in range(2, len(basic_stats_this_team_rows)):#[2,3,4,5,6,8,9,10,11,12,13,14]:
                    if (not(('Reserves' in basic_stats_this_team_rows[i].get_text()) or ('Team Totals' in basic_stats_this_team_rows[i].get_text()))):
#                        print(basic_stats_this_team_rows[i].get_text())
                        this_player_dict = {}
                        this_player_basic_stat_row = basic_stats_this_team_rows[i]
                        try:
                            playerID = this_player_basic_stat_row.find('th').find('a').get('href').split('/')[3][:-5]
                        except AttributeError:
                            print(this_player_basic_stat_row)
                            break

                        this_player_dict['playerID'] = playerID
                        this_player_dict['gameID'] = gameID
                        
                        if (theside == 0):
                            this_player_dict['vis_or_home'] = 0
                        elif (theside == 8):
                            this_player_dict['vis_or_home'] = 1
                        else:
                            print('neither visitor nor home?')
                            
                        if (i <= 6):
                            this_player_dict['starter'] = True
                        else:
                            this_player_dict['starter'] = False

                        this_player_basic_stats = this_player_basic_stat_row.findAll('td')

                        for j in range(0, len(this_player_basic_stats)):
                            if (j == 0):
                                try:
                                    itsvalue = datetime.strptime(this_player_basic_stats[j].get_text(), '%M:%S')
                                except ValueError:
                                    pass
                                itsvalue = pandas.to_datetime(this_player_basic_stats[j].get_text(), format='%M:%S', errors='coerce')
                            else:
                                itsvalue = pandas.to_numeric(this_player_basic_stats[j].get_text(), errors='coerce')
                            this_player_dict[this_player_basic_stats[j].get('data-stat')] = itsvalue
                        #print(this_player_dict)
                        if ('reason' not in this_player_dict.keys()):
                            this_game_record.append(this_player_dict)
            
            this_game_df = pandas.DataFrame.from_records(this_game_record)
            all_games_df = all_games_df.append(this_game_df, sort=True)
            
print('saving all games metadata to file...')
all_games_meta_df = pandas.DataFrame.from_records(all_games_meta_records)
all_games_meta_df = all_games_meta_df.set_index('gameID')

meta_column_order = ['visiting_team', 'visiting_record_pre_wins', 'visiting_record_pre_losses', 'visitng_pre_win_pct']
meta_column_order += ['home_team', 'home_record_pre_wins', 'home_record_pre_losses', 'home_pre_win_pct']
meta_column_order += ['datetime', 'arena', 'city', 'state', 'visiting_score', 'home_score']#all_games_meta_df = all_games_meta_df[meta_column_order]

all_games_meta_df = all_games_meta_df[meta_column_order]
all_games_meta_df.to_csv(outdir+'games_meta_{0:}.csv'.format(season), encoding='utf-8')


print('saving all game logs to file...')
all_games_df = all_games_df.set_index(['gameID', 'playerID'])
column_order = ['mp', 'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast']
column_order += ['stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus', 'vis_or_home', 'starter']
all_games_df = all_games_df[column_order]
all_games_df.to_csv(outdir+'game_logs_{0:}.csv'.format(season), encoding='utf-8')

print('Done!')


finding games for ORL...
	https://www.basketball-reference.com/boxscores/201910230ORL.html


/home/idies/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in long_scalars
/home/idies/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in long_scalars


	https://www.basketball-reference.com/boxscores/201910300ORL.html
	https://www.basketball-reference.com/boxscores/201911010ORL.html
	https://www.basketball-reference.com/boxscores/201911020ORL.html
	https://www.basketball-reference.com/boxscores/201911080ORL.html
	https://www.basketball-reference.com/boxscores/201911100ORL.html
	https://www.basketball-reference.com/boxscores/201911130ORL.html
finding games for PHI...
	https://www.basketball-reference.com/boxscores/201910230PHI.html
	https://www.basketball-reference.com/boxscores/201910300PHI.html
	https://www.basketball-reference.com/boxscores/201911100PHI.html
	https://www.basketball-reference.com/boxscores/201911120PHI.html
finding games for PHO...
	https://www.basketball-reference.com/boxscores/201910230PHO.html
	https://www.basketball-reference.com/boxscores/201910260PHO.html
	https://www.basketball-reference.com/boxscores/201910280PHO.html
	https://www.basketball-reference.com/boxscores/201911040PHO.html
	https://www.basketball-re

In [3]:
#zdf = pandas.read_csv(outdir+'games_meta_{0}.csv'.format(season), encoding='utf-8', low_memory=False)
#zdf = zdf.set_index('gameID')
#zdf

In [4]:
#df = pandas.read_csv(outdir+'game_logs_{0:}.csv'.format(season), encoding='utf-8', low_memory=False)
#df = df.set_index(['gameID', 'playerID'])
#df

for x in ['visitor', 'home']:
    if (x == 'visitor'):
        thisteam = thetables[0].get_text()[0:thetables[0].get_text().find("(")-1]
        basic_stats_rows = thetables[6].findAll('tr')
        advanced_stats_rows = thetables[7].findAll('tr')
    elif (x == 'home'):
        thisteam = thetables[8].get_text()[0:thetables[0].get_text().find("(")-1]
        basic_stats_rows = thetables[14].findAll('tr')
        advanced_stats_rows = thetables[15].findAll('tr')
    else:
        print('not visitor or home')
    
    this_game_record_adv = []
    for i in range(2, len(advanced_stats_rows)):
        #print(i) 
        #pprint(advanced_stats_rows[i])
        #print('\n')
        this_player_dict_adv = {}
        thisplayer_adv = advanced_stats_rows[i].findAll('th')[0].get_text()
        
        if (thisplayer_adv not in (['Reserves', 'Team Totals'])):
            linkedpage_adv = advanced_stats_rows[i].findAll('th')[0].find('a').get('href')
            playerid_adv = re.split('/', linkedpage_adv)[3][0:re.split('/', linkedpage_adv)[3].find('.')]
            this_player_dict_adv['playerID'] = playerid_adv
            this_player_dict_adv['gameID'] = gameID
            this_player_dict_adv['team'] = thisteam
            this_player_stats_adv = advanced_stats_rows[i].findAll('td')
            for k in range(0, len(this_player_stats_adv)):
                #print(k)
                #pprint(this_player_stats_adv[k].get('data-stat'))
                #pprint(this_player_stats_adv[k].get_text())
                #print('\n')
            #    pprint(this_player_stats_adv[k])
                this_player_dict_adv[this_player_stats_adv[k].get('data-stat')] = this_player_stats_adv[k].get_text()
            #    pprint(this_player_dict_adv.values())
            #pprint(this_player_dict_adv)
            if ('Did Not Play' not in this_player_dict_adv.values()):
                this_game_record_adv.append(this_player_dict_adv)
    this_game_adv_df = pandas.DataFrame.from_records(this_game_record_adv)
    
    games_adv_df = games_df.append(this_game_adv_df, sort=True)
    #pprint(advanced_stats_rows)
games_adv_df = games_adv_df.set_index(['gameID', 'playerID'])
games_adv_df